<a href="https://colab.research.google.com/github/Dipeshpal/Intent-Classification-small-transformers/blob/main/Intent_Classification_small_Jarvis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part A: Prepare a 🤗 Hugging Face Dataset from Data from CSV Format

In [ ]:
!pip install datasets transformers shutup ipython-autotime evaluate shutup --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
dataset_file = 'dataset-filtered-renamed.csv'
model_dipesh = 'Intent-Classification-small'
tokenizer_dipesh = 'Intent-Classification-small'
dataset_dipesh = 'Intent-Classification-small'
per_device_train_batch_size=64
per_device_eval_batch_size=64
num_train_epochs=8
base_model = 'joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english'

time: 690 µs (started: 2024-09-07 14:57:15 +00:00)


In [ ]:
%load_ext autotime
import shutup
shutup.please()

time: 1.74 ms (started: 2024-09-07 14:46:13 +00:00)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

pd.set_option('max_colwidth', 400)

time: 1.89 s (started: 2024-09-07 14:46:18 +00:00)


In [ ]:
base_path = '/content/drive/MyDrive/ExcelR_GenAI_Practicals/1-Intent-Classification-small-transformers-main/dataset_folder/data_BALANCE_DATA-True/'

time: 569 µs (started: 2024-09-07 14:56:05 +00:00)


In [ ]:
dataset = load_dataset("csv", data_files=base_path+dataset_file)

Generating train split: 0 examples [00:00, ? examples/s]

time: 1.42 s (started: 2024-09-07 14:57:22 +00:00)


In [ ]:
dataset = dataset["train"].shuffle(seed=42)

time: 8.16 ms (started: 2024-09-07 14:57:32 +00:00)


In [ ]:
# dataset = dataset.rename_column(
#     original_column_name="labels", new_column_name="label"
# )
dataset = dataset.remove_columns('labels')

time: 2.11 ms (started: 2024-09-07 14:57:39 +00:00)


In [ ]:
dataset = dataset.train_test_split(train_size=0.9, seed=42)
# Rename the default "test" split to "validation"
dataset["validation"] = dataset.pop("test")

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 684
    })
})

time: 17.9 ms (started: 2024-09-07 14:57:46 +00:00)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model)

dataset = dataset.map(lambda examples: tokenizer(examples["text"], return_tensors="np"), batched=True)

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/6153 [00:00<?, ? examples/s]

Map:   0%|          | 0/684 [00:00<?, ? examples/s]

time: 9.1 s (started: 2024-09-07 14:57:54 +00:00)


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask'],
        num_rows: 684
    })
})

time: 2.94 ms (started: 2024-09-07 14:58:09 +00:00)


### Generating id2label, label2id and num_labels

In [ ]:
from datasets import Dataset, Value, ClassLabel, Features

# Creating a ClassLabel Object
df = dataset["train"].to_pandas()
labels = df['intent'].unique().tolist()
ClassLabels = ClassLabel(num_classes=len(labels), names=labels)

# Mapping Labels to IDs
def map_label2id(example):
    example['label'] = ClassLabels.str2int(example['intent'])
    return example

dataset = dataset.map(map_label2id, batched=True)

# Casting label column to ClassLabel Object
dataset = dataset.cast_column('label', ClassLabels)
dataset

Map:   0%|          | 0/6153 [00:00<?, ? examples/s]

Map:   0%|          | 0/684 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6153 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/684 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 214 ms (started: 2024-09-07 14:58:21 +00:00)


In [ ]:
# dataset = dataset.align_labels_with_mapping(label2id, "label")

time: 357 µs (started: 2023-01-27 22:08:06 +00:00)


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 2.61 ms (started: 2024-09-07 14:58:31 +00:00)


In [ ]:
dataset['train'][0]

{'text': 'Thank you, goodbye',
 'intent': 'goodbye',
 'input_ids': [101, 4067, 2017, 1010, 9119, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1],
 'label': 0}

time: 3.51 ms (started: 2024-09-07 14:58:36 +00:00)


In [ ]:
#dataset.push_to_hub(f"dipesh/{dataset_dipesh}")

time: 343 µs (started: 2024-09-07 14:59:00 +00:00)


# PART B: Hugging Face Tokenization: Use a Pre-trained Tokenizer or Train a New Tokenizer from scratch?

As suggested in the official documentation, if a language model is not available in the language you are interested in, or if your corpus is very different from the one your language model was trained on, you will most likely want to retrain the model from scratch using a tokenizer adapted to your data. That will require training a new tokenizer on your dataset.

## Using Pretrained

In [ ]:
from transformers import AutoTokenizer

context_length = 40
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)

outputs = pretrained_tokenizer(
    dataset["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=False,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print("vocab_size: ", len(pretrained_tokenizer))

Input IDs length: 2
Input chunk lengths: [6, 9]
vocab_size:  30522
time: 327 ms (started: 2024-09-07 14:59:10 +00:00)


In [ ]:
txt = "how are you"
tokens = pretrained_tokenizer(txt)['input_ids']
print(tokens)

[101, 2129, 2024, 2017, 102]
time: 1e+03 µs (started: 2024-09-07 14:59:18 +00:00)


In [ ]:
converted = pretrained_tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 5.73 ms (started: 2024-09-07 14:59:21 +00:00)


## Train a new tokenizer

In [ ]:
def get_training_corpus():
    batch_size = 1000
    return (
        dataset["train"][i : i + batch_size]["text"]
        for i in range(0, len(dataset["train"]), batch_size)
    )
training_corpus = get_training_corpus()

time: 783 µs (started: 2024-09-07 14:59:28 +00:00)


In [ ]:
for data in get_training_corpus():
    print(len(data))

1000
1000
1000
1000
1000
1000
153
time: 77.2 ms (started: 2024-09-07 14:59:32 +00:00)


### NOTE: Now that we have our corpus in the form of an iterator of batches of texts, we are ready to train a new tokenizer. To do this, we first need to load the tokenizer we want to pair with our model

### NOTE: Even though we are going to train a new tokenizer, it’s a good idea to do this to avoid starting entirely from scratch. This way, we won’t have to specify anything about the tokenization algorithm or the special tokens we want to use; our new tokenizer will be exactly the same as facebook/bart-large-mnli, and the only thing that will change is the vocabulary, which will be determined by the training on our corpus.

In [ ]:
pretrained_tokenizer = AutoTokenizer.from_pretrained(base_model)
vocab_size = len(pretrained_tokenizer)
tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus,vocab_size)

time: 663 ms (started: 2024-09-07 14:59:38 +00:00)


In [ ]:
print(tokenizer.eos_token_id, ",", tokenizer.vocab_size)

None , 3426
time: 1.18 ms (started: 2024-09-07 14:59:42 +00:00)


In [ ]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print(tokens)

[2, 180, 145, 88, 3]
time: 2.03 ms (started: 2024-09-07 14:59:47 +00:00)


In [ ]:
converted = tokenizer.convert_ids_to_tokens(tokens)
print(converted)

['[CLS]', 'how', 'are', 'you', '[SEP]']
time: 637 µs (started: 2024-09-07 14:59:51 +00:00)


In [ ]:
print("pretrained_tokenizer-")
print(pretrained_tokenizer)

print("New tokenizer-")
print(tokenizer)

pretrained_tokenizer-
DistilBertTokenizerFast(name_or_path='joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
New tokenizer-
DistilBertTokenizerFast

In [ ]:
print("New tokenizer: ", len(tokenizer.tokenize(txt)))
print("Pretrained tokenizer: ", len(pretrained_tokenizer.tokenize(txt)))

New tokenizer:  3
Pretrained tokenizer:  3
time: 2.82 ms (started: 2024-09-07 15:00:07 +00:00)


### Saving tokenizer in local

In [ ]:
path=f"/"
file_name=model_dipesh
tokenizer.save_pretrained(path+file_name)

('/Intent-Classification-small/tokenizer_config.json',
 '/Intent-Classification-small/special_tokens_map.json',
 '/Intent-Classification-small/vocab.txt',
 '/Intent-Classification-small/added_tokens.json',
 '/Intent-Classification-small/tokenizer.json')

time: 7.86 ms (started: 2024-09-07 15:00:11 +00:00)


### Loading tokenizer from local

In [ ]:
loaded_tokenizer = AutoTokenizer.from_pretrained(path+file_name)

time: 5.95 ms (started: 2024-09-07 15:00:54 +00:00)


In [ ]:
txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)

trained tokenizer: [2, 180, 145, 88, 3]
loaded tokenizer: [2, 180, 145, 88, 3]
time: 4.62 ms (started: 2024-09-07 15:00:59 +00:00)


In [ ]:
#tokenizer.push_to_hub(f"dipesh/{tokenizer_dipesh}")

time: 369 µs (started: 2024-09-07 15:01:18 +00:00)


You can then load the tokenizer from anywhere with the from_pretrained() method:

In [ ]:
downloaded_tokenizer = AutoTokenizer.from_pretrained(f"dipesh/{model_dipesh}")

txt = "how are you"
tokens = tokenizer(txt)['input_ids']
print("trained (New) tokenizer:", tokens)
tokens = loaded_tokenizer(txt)['input_ids']
print("loaded tokenizer:", tokens)
tokens = downloaded_tokenizer(txt)['input_ids']
print("downloaded tokenizer:", tokens)

tokenizer_config.json:   0%|          | 0.00/415 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/21.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/74.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

trained (New) tokenizer: [2, 180, 145, 88, 3]
loaded tokenizer: [2, 180, 145, 88, 3]
downloaded tokenizer: [2, 180, 145, 88, 3]
time: 2.5 s (started: 2024-09-07 15:01:24 +00:00)


# PART C: Training a model

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, DistilBertTokenizerFast, RobertaTokenizer
from transformers import TFAutoModelForSequenceClassification, TFDistilBertForSequenceClassification, \
    TFRobertaForSequenceClassification, TFAutoModel, AutoModel, AutoModelForSequenceClassification
from sklearn import preprocessing
from datasets import load_dataset

time: 3.51 s (started: 2024-09-07 15:01:33 +00:00)


### Loading Dataset

In [ ]:
f'dipesh/{dataset_dipesh}'

'dipesh/Intent-Classification-small'

time: 3 ms (started: 2024-09-07 15:01:41 +00:00)


In [ ]:
dataset = load_dataset(f'dipesh/Intent-Classification-small')
dataset

Generating train split:   0%|          | 0/6153 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/684 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 8.84 s (started: 2024-09-07 15:01:50 +00:00)


### Creating Datacollector and evaluator

In [ ]:
from transformers import DataCollatorWithPadding
import evaluate


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

time: 3.55 s (started: 2024-09-07 15:02:30 +00:00)


In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

time: 784 µs (started: 2024-09-07 15:02:44 +00:00)


### Loading Model

In [ ]:
def id2label_gen(id):
    return ClassLabels.int2str(id)

def label2id_gen(label):
    return ClassLabels.str2int(label)

total_labels = ClassLabels.num_classes

print(id2label_gen(3), label2id_gen(id2label_gen(3)),  "total_labels: ", total_labels)

covid cases 3 total_labels:  21
time: 2.77 ms (started: 2024-09-07 15:02:48 +00:00)


In [ ]:
id2label = {i: ClassLabels.int2str(i) for i in range(total_labels)}

label2id = {ClassLabels.int2str(i): i for i in range(total_labels)}
id2label, label2id

({0: 'goodbye',
  1: 'volume control',
  2: 'play games',
  3: 'covid cases',
  4: 'open website',
  5: 'tell me joke',
  6: 'play on youtube',
  7: 'places near me',
  8: 'greet and hello hi kind of things, general check in',
  9: 'asking time',
  10: 'asking date',
  11: 'tell me news',
  12: 'asking weather',
  13: 'download youtube video',
  14: 'what can you do',
  15: 'take screenshot',
  16: 'send email',
  17: 'i am bored',
  18: 'click photo',
  19: 'tell me about',
  20: 'send whatsapp message'},
 {'goodbye': 0,
  'volume control': 1,
  'play games': 2,
  'covid cases': 3,
  'open website': 4,
  'tell me joke': 5,
  'play on youtube': 6,
  'places near me': 7,
  'greet and hello hi kind of things, general check in': 8,
  'asking time': 9,
  'asking date': 10,
  'tell me news': 11,
  'asking weather': 12,
  'download youtube video': 13,
  'what can you do': 14,
  'take screenshot': 15,
  'send email': 16,
  'i am bored': 17,
  'click photo': 18,
  'tell me about': 19,
  'send 

time: 4.76 ms (started: 2024-09-07 15:02:53 +00:00)


In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# If not loaded then load tokenizer by uncommenting below line
# tokenizer = AutoTokenizer.from_pretrained("dipesh/Intent-Classification-Commands-balanced-ds-large")
model = AutoModelForSequenceClassification.from_pretrained(base_model, num_labels=total_labels, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)
# model = AutoModelForSequenceClassification.from_pretrained(base_model, ignore_mismatched_sizes=True)

config.json:   0%|          | 0.00/3.85k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at joaobarroca/distilbert-base-uncased-finetuned-massive-intent-detection-english and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([60]) in the checkpoint and torch.Size([21]) in the model instantiated
- classifier.weight: found shape torch.Size([60, 768]) in the checkpoint and torch.Size([21, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


time: 25.2 s (started: 2024-09-07 15:03:01 +00:00)


### Creating tokenized dataset

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)


tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/6153 [00:00<?, ? examples/s]

Map:   0%|          | 0/684 [00:00<?, ? examples/s]

time: 493 ms (started: 2024-09-07 15:03:39 +00:00)


In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 6153
    })
    validation: Dataset({
        features: ['text', 'intent', 'input_ids', 'attention_mask', 'label'],
        num_rows: 684
    })
})

time: 3.69 ms (started: 2024-09-07 15:03:44 +00:00)


### Finetuning Model

In [ ]:
config = {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": id2label,
  "initializer_range": 0.02,
  "label2id": label2id,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": False,
  "tie_weights_": True,
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "vocab_size": 30522
}

time: 1.09 ms (started: 2024-09-07 15:03:52 +00:00)


In [ ]:
import json

with open(path+file_name+'/config.json', "w") as f:
    json.dump(config, f)

time: 1.9 ms (started: 2024-09-07 15:03:57 +00:00)


In [ ]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained(base_model)
config.update({"id2label": id2label})
config.update({"label2id": label2id})
# Push the config to your namespace with the name "my-finetuned-bert".
# config.push_to_hub(f"dipesh/{model_dipesh}")

time: 245 ms (started: 2024-09-07 15:04:18 +00:00)


In [ ]:
# import os
# os.system('git init /content/Intent-Classification-small')

time: 378 µs (started: 2023-01-27 22:09:59 +00:00)


In [ ]:
output_dir_model_hf_name=f"dipesh/{model_dipesh}"

training_args = TrainingArguments(
    output_dir=output_dir_model_hf_name,
    learning_rate=2e-5,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.215064,0.334795
2,No log,1.506059,0.595029
3,No log,1.046894,0.744152
4,No log,0.668882,0.862573
5,No log,0.532049,0.885965
6,1.503100,0.438450,0.900585
7,1.503100,0.366948,0.922515
8,1.503100,0.352675,0.925439


TrainOutput(global_step=776, training_loss=1.1247747558908365, metrics={'train_runtime': 142.482, 'train_samples_per_second': 345.475, 'train_steps_per_second': 5.446, 'total_flos': 202854048342444.0, 'train_loss': 1.1247747558908365, 'epoch': 8.0})

time: 2min 33s (started: 2024-09-07 15:04:29 +00:00)


# Inference

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=f"./{output_dir_model_hf_name}")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


time: 78.2 ms (started: 2024-09-07 15:16:55 +00:00)


In [ ]:
classifier("click photo")

[{'label': 'click photo', 'score': 0.8452701568603516}]

time: 268 ms (started: 2024-09-07 15:17:11 +00:00)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained(
    f"dipesh/{tokenizer_dipesh}")
model = AutoModelForSequenceClassification.from_pretrained(
    f"dipesh/{model_dipesh}")
classifier_return_all_scores = pipeline('text-classification', model=model, tokenizer=tokenizer, return_all_scores=True )
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


time: 57.9 ms (started: 2024-09-07 15:17:34 +00:00)


In [ ]:
sequence = "make me laugh"

# Classify the sequence
results = classifier(sequence)

# Print the results
print(results)

[{'label': 'tell me joke', 'score': 0.941535234451294}]
time: 43.9 ms (started: 2024-09-07 15:18:16 +00:00)


In [ ]:
sequence = "make me laugh"

# Classify the sequence
results = classifier_return_all_scores(sequence)

# Print the results
print(results)

[[{'label': 'goodbye', 'score': 0.0032277624122798443}, {'label': 'volume control', 'score': 0.00327436625957489}, {'label': 'play games', 'score': 0.002495155204087496}, {'label': 'covid cases', 'score': 0.0011273154523223639}, {'label': 'open website', 'score': 0.003914643079042435}, {'label': 'tell me joke', 'score': 0.941535234451294}, {'label': 'play on youtube', 'score': 0.0014629514189437032}, {'label': 'places near me', 'score': 0.0017538181273266673}, {'label': 'greet and hello hi kind of things, general check in', 'score': 0.0022457384038716555}, {'label': 'asking time', 'score': 0.002281232038512826}, {'label': 'asking date', 'score': 0.000989614869467914}, {'label': 'tell me news', 'score': 0.007165633607655764}, {'label': 'asking weather', 'score': 0.0019369465298950672}, {'label': 'download youtube video', 'score': 0.004169626161456108}, {'label': 'what can you do', 'score': 0.0011664705816656351}, {'label': 'take screenshot', 'score': 0.000876172271091491}, {'label': 'se

time: 55.7 ms (started: 2023-01-27 22:12:55 +00:00)
